## Chat with your Database

Imagine chatting with your database using just a few lines of code. Sounds unbelievable, right? Well, believe it! We'll show you how you can effortlessly chat with a huge database containing 10 million business records—all with just a few lines of SuperDuperDB code.

Let's do this with SuperDuperDB's user-friendly approach. You can control the low level code while enjoying writing the high level code! Whether you're using Snowflake or any other SQL dataset, we've got your back.

Here's the simplicity of it:
1. Connect using your URI (works with any SQL Database).
2. Specify your Database/Table Name.
3. Craft a query in plain English.

You'll not only get results but also clear explanations!

Let SuperDuperDB and OpenAI do the heavy lifting—all within a single prompt. Stay tuned for more exciting features, including prompt chaining!

Bring the power of AI into your database effortlessly! 

Let's bring AI into your database! 🚀

In [ ]:
from superduperdb import superduper
from superduperdb.backends.ibis.query import RawSQL

user = "superduperdb"
password = "superdhardpassword"
account = "ORGANIZATIONID-USERID"  # ORGANIZATIONID-USERID
database = "FREE_COMPANY_DATASET/PUBLIC"  # DATABASE/SCHEMA 

# Here we are using Snowflake FREE_COMPANY_DATASET with 10 million company data
snowflake_uri = f"snowflake://{user}:{password}@{account}/{database}"

# Let's superduper your database
db = superduper(
    snowflake_uri,
    metadata_store='mongodb://localhost:27017/documents', # We need a persistent metadata store to store everything. It could be anything including your own database.
)

In [10]:
import os

from superduperdb.ext.openai import OpenAIChatCompletion
from IPython.display import Markdown

# Add your OPEN_AI_API_KEY or keep it in your os.environ, we will pick it up from environment
os.environ['OPENAI_API_KEY'] = 'sk-XXXXXXXXXXXXXXXXXXXXXV'

In [35]:
def chat_with_your_database(table_name, query):
    # Define the search parameters
    search_term = f'Write me a SQL query for the table named {table_name}. The query is: {query}'

    # Define the prompt for the OpenAIChatCompletion model
    prompt = (
        f'I will give you a CSV of a SQL table. Please write an SQL query based on the context.\n'
        'Context: {context}\n\n'
        'Provide only the SQL query. Add limit 5 in the SQL query. Do not output any word except SQL Query.'
        'Here\'s the CSV file:\n'
    )

    # Add the OpenAIChatCompletion instance to the database
    db.add(OpenAIChatCompletion(model='gpt-3.5-turbo', prompt=prompt))
    
    # Use the OpenAIChatCompletion model to predict the next query based on the provided context
    output, context = db.predict(
        model_name='gpt-3.5-turbo',
        input=search_term,
        context_select=db.execute(RawSQL(f'SELECT * FROM {table_name} LIMIT 10')).as_pandas().to_csv()
    )
    
    try:
        # Attempt to execute the predicted SQL query and retrieve the result as a pandas DataFrame
        query_result = db.execute(RawSQL(output.content)).as_pandas()
    except:
        # If an exception occurs, provide a message to guide the user on adjusting their query
        query_result = "Please edit your query based on the database so that we can find you a suitable result. Please check your schema if you encounter issues."

    return query_result


In [36]:
def explain_the_result(query_result):
    # Define the search parameters
    search_term = f'Find business insights from it {query_result.to_csv()}'

    # Define the prompt for the OpenAIChatCompletion model
    prompt = (
        f'You will be acting as a Business Analyst. Your goal is to give correct business insights from the given CSV content. Do not output the same CSV file again. Do not write the SQL query again. Write a summary finally.\n'
        'Context: {context}'
        'Here\'s the CSV file for you to analyze:\n'
    )
    
    # Add the OpenAIChatCompletion instance to the database
    db.add(OpenAIChatCompletion(model='gpt-3.5-turbo', prompt=prompt))
    
    # Use the OpenAIChatCompletion model to predict insights based on the provided context
    output, context = db.predict(
        model_name='gpt-3.5-turbo',
        input=search_term,
    )
    
    try:
        # Attempt to format the predicted output as Markdown
        query_result = Markdown(output.content)
    except:
        # If an exception occurs, provide a message to guide the user on adjusting their input
        query_result = "Please edit your input based on the dataset so that we can find you a suitable output. Please check your data if you encounter issues."

    return query_result

In [39]:
# Assuming you have already defined and implemented the chat_with_your_database function

table_name = "FREECOMPANYDATASET"
query = "Find some company in germany in berlin"

# Call the chat_with_your_database function to execute the query
result = chat_with_your_database(table_name, query)

# Print or further process the query result
print(result)


,COUNTRY,FOUNDED,ID,INDUSTRY,LINKEDIN_URL,LOCALITY,NAME,REGION,SIZE,WEBSITE
0,germany,NaN,yoga-thinking,management consulting,linkedin.com/company/yoga-thinking,berlin,yoga thinking,berlin,1-10,yoga-thinking.com
1,germany,2020.0,local-gay-guide,None,linkedin.com/company/local-gay-guide,berlin,local gay guide,berlin,1-10,localgayguide.com
2,germany,NaN,schrobsdorff-bau-ag,construction,linkedin.com/company/schrobsdorff-bau-ag,berlin,schrobsdorff bau ag,berlin,51-200,schrobsdorff.ag
3,germany,NaN,efendi-optik-gmbh,None,linkedin.com/company/efendi-optik-gmbh,berlin,efendi optik gmbh,berlin,1-10,efendi-optik.de
4,germany,2022.0,center-for-brain-leadership,professional training & coaching,linkedin.com/company/center-for-brain-leadership,berlin,center for brain leadership,berlin,1-10,brainleader.io


In [41]:
# Assuming you have already defined and implemented the explain_the_result function

# Call the explain_the_result function to analyze and explain the business insights
analysis_result = explain_the_result(query_result=result)

# Print or further process the analysis result
print(analysis_result)

Here are some business insights from the given data:

1. Yoga Thinking is a management consulting company based in Berlin, Germany. It was founded in an unspecified year and has a team size of 1-10 employees. You can find more information about them on their website - yoga-thinking.com.

2. Local Gay Guide is a German business founded in 2020. While the industry of this company is not specified, it is based in Berlin and has a team size of 1-10 employees. You can find more details about them on their LinkedIn profile or their website - localgayguide.com.

3. Schrobsdorff Bau AG is a construction company headquartered in Berlin, Germany. The year of its founding is unknown, but it has a team size of 51-200 employees. You can find more information about the company on its LinkedIn page - linkedin.com/company/schrobsdorff-bau-ag.

4. Efendi Optik GmbH is a Berlin-based company in the optical industry. The founding year and team size are not specified. For more details about the company, you can visit their LinkedIn profile - linkedin.com/company/efendi-optik-gmbh or their website - efendi-optik.de.

5. Center for Brain Leadership is a professional training and coaching company. It is located in Berlin, Germany and was founded in 2022. The company has a team size of 1-10 employees. To learn more about them, you can visit their LinkedIn page - linkedin.com/company/center-for-brain-leadership or their website - brainleader.io.

Please note that some information, such as founding year and team size, may not be available for all businesses in the given dataset.

## Unlocking Limitless Potential! 

This is just the beginning – feel free to customize prompts for your dataset. One secret tips: Mentioning your database schema in the `chat_your_database` function enhances accuracy by a few mile. Another one is giving more data to it. Anyway, it's yours. Play with it. The better you prompt, the better result you get. This prompt of us is just a simple one works for everything! Your journey with SuperDuperDB is in your hands now. Let the exploration begin!